In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
# 📁 STEP 2: Mount Google Drive (if dataset is there)
from google.colab import drive
drive.mount('/content/drive')


IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/Brain Tumor Segmentation/Testing'

CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']
data = []
labels = []

# ✅ Load and resize images
for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except Exception as e:
            pass

# Preprocessing
data = np.array(data) / 255.0
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# Function to make predictions
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label = lb.inverse_transform(prediction)
    return label[0]

# Create Gradio interface
interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type='pil'),
                         outputs=gr.Label(num_top_classes=4),
                         title="Brain Tumor Classifier",
                         description="Upload an image to classify a brain tumor as glioma, meningioma, notumor, or pituitary.")
interface.launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 58s 717ms/step - accuracy: 0.5137 - loss: 1.1270 - val_accuracy: 0.2357 - val_loss: 1.9053
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 76s 637ms/step - accuracy: 0.6178 - loss: 0.9215 - val_accuracy: 0.2357 - val_loss: 3.8677
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 83s 661ms/step - accuracy: 0.6789 - loss: 0.7784 - val_accuracy: 0.2357 - val_loss: 6.8722
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 81s 649ms/step - accuracy: 0.7035 - loss: 0.6947 - val_accuracy: 0.2357 - val_loss: 8.3480
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 43s 659ms/step - accuracy: 0.7357 - loss: 0.6388 - val_accuracy: 0.2357 - val_loss: 9.7029
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 43s 658ms/step - accuracy: 0.7156 - loss: 0.6406 - val_accuracy: 0.2548 - val_loss: 4.9450
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 42s 640ms/step - accuracy: 0.7651 - loss: 0.5830 - val_accuracy: 0.2471 - val_loss: 4.5319
Epoch 8/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 41s 618ms/step - accuracy: 0.7538 - loss: 0.5827 - val_accu